In [ ]:
pip install neattext

In [ ]:
import re
import string
import nltk
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

import neattext.functions as nfx
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

from nltk.corpus import stopwords
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Embedding, Dense, LSTM, Dropout, Conv1D, GlobalMaxPooling1D, MaxPooling1D,  Activation
from keras.callbacks import Callback, EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [ ]:
import pandas as pd
from pandas import read_excel
import numpy as np
import re
from re import sub
import multiprocessing
#from unidecode import unidecode
import os
from time import time 
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout,Activation,Embedding,Flatten,Bidirectional,MaxPooling2D, Conv1D, MaxPooling1D
#from keras.optimizers import SGD,Adam
from keras import regularizers
from keras.preprocessing.text import Tokenizer
#from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
import h5py
import csv
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pip install openpyxl

In [ ]:
traind = pd.read_excel("/content/nlpaug_4augmented_train.xlsx")
vald = pd.read_excel("/content/nlpaug_4augmented_val.xlsx")
testd = pd.read_excel("/content/nlpaug_4augmented_test.xlsx")

In [ ]:
traind["Review"]

In [ ]:
train_I=traind["Review"].to_numpy()
val_I=vald["Review"].to_numpy()
test_I=testd["Review"].to_numpy()
train_L=traind["Label"].to_numpy()
val_L=vald["Label"].to_numpy()
test_L=testd["Label"].to_numpy()

In [ ]:
import keras
from keras.utils.np_utils import to_categorical

In [ ]:
training_sentences = []
testing_sentences = []
val_sentences = []
train_sentences = list(train_I)
train_labels = train_L
for i in range(len(train_sentences)): 
    #print(train_sentences[i])
    x=str(train_sentences[i])
    training_sentences.append(x)
training_sentences=np.array(training_sentences)

test_sentences=list(test_I)
test_labels=test_L

for i in range(len(test_sentences)): 
    x=str(test_sentences[i])
    testing_sentences.append(x)
testing_sentences=np.array(testing_sentences)

validation_sentences=list(val_I)
val_labels=val_L

for i in range(len(validation_sentences)): 
    x=str(validation_sentences[i])
    val_sentences.append(x)
val_sentences=np.array(val_sentences)


train_labels=to_categorical(train_labels)
test_labels=to_categorical(test_labels)
val_labels=to_categorical(val_labels)

# print("Training Set Length: "+str(len(train1)))
# print("Testing Set Length: "+str(len(test1)))
print("training_sentences shape: "+str(training_sentences.shape))
print("testing_sentences shape: "+str(testing_sentences.shape))
print("validation_sentences shape: "+str(val_sentences.shape))
print("train_labels shape: "+str(train_labels.shape))
print("test_labels shape: "+str(test_labels.shape))
print("val_labels shape: "+str(val_labels.shape))

In [ ]:
vocab_size = 25000
embedding_dim = 300
max_length = 512
trunc_type='post'
oov_tok = "<OOV>"

In [ ]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
print(len(word_index))
print("Word index length:"+str(len(tokenizer.word_index)))
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)


test_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(test_sequences,maxlen=max_length)

val_sequences = tokenizer.texts_to_sequences(val_sentences)
val_padded = pad_sequences(val_sequences,maxlen=max_length)

In [ ]:
print("Sentence :--> \n")
print(training_sentences[2]+"\n")
print("Sentence Tokenized and Converted into Sequence :--> \n")
print(str(sequences[2])+"\n")
print("After Padding the Sequence with padding length 100 :--> \n")
print(padded[2])

In [ ]:
with tf.device('/gpu:0'):
    import keras
    from keras.layers import Conv1D,MaxPooling1D,Flatten,Dense,Dropout,Embedding
    max_words_len=350
    encoder_input=keras.Input(shape=(max_words_len,),)
    encoder_embedding = Embedding(vocab_size, embedding_dim, input_length=max_length)
    encoder_embedding_layer=encoder_embedding(encoder_input)
    conv1D_1=Conv1D(512, 4, activation='relu',name='con1')(encoder_embedding_layer)
    maxPool1D_1=MaxPooling1D(pool_size=2,name='maxpool1')(conv1D_1)
    conv1D_2=Conv1D(256, 3, activation='relu',name='con2')(maxPool1D_1)
    maxPool1D_2=MaxPooling1D(pool_size=2,name='maxpool2')(conv1D_2)
    conv1D_3=Conv1D(128, 2, activation='relu',name='con3')(maxPool1D_2)
    maxPool1D_3=MaxPooling1D(pool_size=2,name='maxpool3')(conv1D_3)
    # Apply Bidirectional LSTM over embedded inputs
    lstm_outs = tf.keras.layers.Bidirectional(
        keras.layers.LSTM(100, return_sequences=True)
    )(maxPool1D_3)

    # Attention Mechanism - Generate attention vectors
    input_dim = int(lstm_outs.shape[2])
    permuted_inputs = keras.layers.Permute((2, 1))(lstm_outs)
    attention_vector = keras.layers.TimeDistributed(keras.layers.Dense(1))(lstm_outs)
    print(attention_vector.shape)
    attention_vector = keras.layers.Reshape((attention_vector.shape[1],))(attention_vector)
    attention_vector = keras.layers.Activation('softmax', name='attention_vec')(attention_vector)
    attention_output = keras.layers.Dot(axes=1)([lstm_outs, attention_vector])
    fc = keras.layers.Dense(100, activation='relu')(attention_output)
    output = keras.layers.Dense(2, activation='softmax')(fc)
    model = keras.Model(inputs=[encoder_input], outputs=output)
    model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer='adam')
    model.summary()

In [ ]:
epoch=15
batch=256

In [ ]:
history=model.fit(padded,train_labels,epochs=epoch,batch_size=batch,validation_data=(val_padded,val_labels),use_multiprocessing=True, workers=8)

In [ ]:
print(history.history.keys())
loss = history.history['loss']
val_loss = history.history['val_loss']
plt.plot(loss)
plt.plot(val_loss)
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['loss', 'val_loss'])
plt.show()

accuracy = history.history['accuracy']
val_accuracy= history.history['val_accuracy']
plt.plot(accuracy)
plt.plot(val_accuracy)
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['accuracy', 'val_accuracy'])
plt.show()

In [ ]:
predict_x=model.predict(testing_padded) 
classes_xy=np.argmax(predict_x,axis=1)
loss_and_metrics = model.evaluate(testing_padded,test_labels,batch_size=batch)
print("The test accuracy is: "+str(loss_and_metrics[1]))

In [ ]:
from collections import Counter

from sklearn.metrics import roc_auc_score
x=roc_auc_score(test_labels,model.predict(testing_padded) )
print(x)

from sklearn.metrics import matthews_corrcoef
y_true=np.argmax(test_labels,axis=1)
classes_x=[-1 if item==0 else item for item in classes_xy]
y_true=[-1 if item==0 else item for item in y_true]
mcc=matthews_corrcoef(y_true,classes_x)
print(mcc)

from sklearn.metrics import confusion_matrix
conf=confusion_matrix(y_true, classes_x)
print(conf[0][0])
print(conf[0][1])
print(conf[1][0])
print(conf[1][1])
cr=classification_report(np.argmax(test_labels,axis=1), classes_xy,digits=4,output_dict=True)
print(cr)

In [ ]:
training_stats=[]
training_stats.append(
        {
            'roc-auc': x,
            'mcc': mcc,
            'conf_0_0': conf[0][0],
            'conf_0_1': conf[0][1],
            'conf_1_0': conf[1][0],
            'conf_1_1': conf[1][1],
            'cr': cr
        }
    )

In [ ]:
id=4
modelname="CNNBiLSTM"

In [ ]:
df = pd.DataFrame(training_stats) 
df2 = pd.DataFrame(history.history) 
df3 = pd.DataFrame(predict_x)
df4 = pd.DataFrame(classes_xy,columns=['Pred'])
df4['True']=np.argmax(test_labels,axis=1)
df5= pd.DataFrame(cr).transpose()
    
# saving the dataframe 
df.to_csv("C:/Users/MSI/Desktop/BFRD/Final/nlpaug/performance"+str(id)+str(modelname)+".csv") 
df2.to_csv("C:/Users/MSI/Desktop/BFRD/Final/nlpaug/history"+str(id)+str(modelname)+".csv") 
df3.to_csv("C:/Users/MSI/Desktop/BFRD/Final/nlpaug/probab"+str(id)+str(modelname)+".csv") 
df4.to_csv("C:/Users/MSI/Desktop/BFRD/Final/nlpaug/predict"+str(id)+str(modelname)+".csv") 
df5.to_csv("C:/Users/MSI/Desktop/BFRD/Final/nlpaug/cr"+str(id)+str(modelname)+".csv") 
model.save("C:/Users/MSI/Desktop/BFRD/Final/nlpaug/model"+str(id)+str(modelname)+".h5")